# **MolecularForecast challenge**
*A guide for starting the challenge*

In [302]:
!pip install rdkit

In [303]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import rdkit
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from rdkit import DataStructs
import pandas as pd
from rdkit.Chem import Descriptors, Lipinski, EState, AllChem, rdMolDescriptors
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import VarianceThreshold

# Opening the dissociation constant dataframe


In [304]:
Kd = pd.read_excel('data/Table-3_train.xlsx')

#Kd.replace(10001, np.nan, inplace=True)
#Kd.to_excel('data/Kd.xlsx', index=False)

#DROP THIS BECAUASE IT IS ALL 10001 :(
Kd.drop(Kd.index[420], inplace=True)

print(Kd)

    Accession Number Entrez Gene Symbol                         Kinase  \
0        NP_055726.3               AAK1                           AAK1   
1        NP_005148.2               ABL1     ABL1(E255K)-phosphorylated   
2        NP_005148.2               ABL1  ABL1(F317I)-nonphosphorylated   
3        NP_005148.2               ABL1     ABL1(F317I)-phosphorylated   
4        NP_005148.2               ABL1  ABL1(F317L)-nonphosphorylated   
5        NP_005148.2               ABL1     ABL1(F317L)-phosphorylated   
6        NP_005148.2               ABL1  ABL1(H396P)-nonphosphorylated   
7        NP_005148.2               ABL1     ABL1(H396P)-phosphorylated   
8        NP_005148.2               ABL1     ABL1(M351T)-phosphorylated   
9        NP_005148.2               ABL1  ABL1(Q252H)-nonphosphorylated   
10       NP_005148.2               ABL1     ABL1(Q252H)-phosphorylated   
11       NP_005148.2               ABL1  ABL1(T315I)-nonphosphorylated   
12       NP_005148.2               ABL

In [305]:


# 'data/Table-2_train.csv' for training
# returns molecules list
def load_csv(file):
    # 1. Load the CSV file with SMILES
    sensitivities = pd.read_csv(file)

    # 2. Convert the SMILES column to a list and create RDKit molecule objects
    smiles_list = sensitivities['SMILES'].tolist()
    return [Chem.MolFromSmiles(smi) for smi in smiles_list]

# 3. Define a function to compute the descriptors for each molecule
def featurize_molecule(mol):
    if mol is None:
        return None
    return [
        Descriptors.MolWt(mol),               # Molecular Weight
        Descriptors.MolLogP(mol),             # LogP
        np.sum(EState.EStateIndices(mol)),       # Electrotopological state: sum of EState indice
        Descriptors.NumHDonors(mol),          # Hydrogen Bond Donors
        Descriptors.NumHAcceptors(mol),       # Hydrogen Bond Acceptors
        Descriptors.TPSA(mol),                # Topological Polar Surface Area
        Descriptors.NumRotatableBonds(mol),   # Number of Rotatable Bonds
        Lipinski.NumAromaticRings(mol),       # Aromaticity: count of aromatic rings

    ]

molecules = load_csv('data/Table-2_train.csv')
# 4. Compute descriptors for each molecule and store them in an array
features = [featurize_molecule(mol) for mol in molecules]
features_array = np.array(features)

# 5. Create a DataFrame where each row is a molecule and each column is a descriptor
basic_cols = [
    "MolWt", "MolLogP", "EStateSum", "NumHDonors", "NumHAcceptors",
    "TPSA", "NumRotBonds", "NumAromaticRings"
]
features_df = pd.DataFrame(features_array, columns=basic_cols)

# 6. Transpose the DataFrame so that rows = parameters, columns = molecules
transposed_df = features_df.T

# 7. (Optional) label the columns to indicate which molecule they refer to
transposed_df.columns = [f"Molecule_{i+1}" for i in range(len(transposed_df.columns))]

# TESTING DATA
molecules_test = load_csv('data/Table-2_test.csv')
# 4. Compute descriptors for each molecule and store them in an array
features_test = [featurize_molecule(mol) for mol in molecules_test]
features_array_test = np.array(features_test)

# 5. Create a DataFrame where each row is a molecule and each column is a descriptor
features_df_test = pd.DataFrame(features_array_test, columns=basic_cols)

# 6. Transpose the DataFrame so that rows = parameters, columns = molecules
transposed_df_test = features_df_test.T

# 7. (Optional) label the columns to indicate which molecule they refer to
transposed_df_test.columns = [f"Molecule_{i+1}" for i in range(len(transposed_df_test.columns))]

# 8. (Optional) inspect the final structure
print(transposed_df_test)

                  Molecule_1  Molecule_2  Molecule_3  Molecule_4  Molecule_5  \
MolWt             569.448000  539.636000  478.664000  543.615000  348.362000   
MolLogP             2.186900    3.615900    5.037800    5.092100    2.980200   
EStateSum         112.027778   91.333333   66.897778   96.416667   57.333333   
NumHDonors          9.000000    2.000000    2.000000    1.000000    1.000000   
NumHAcceptors       7.000000    7.000000    3.000000    7.000000    7.000000   
TPSA              234.460000   94.220000   50.360000   76.620000   84.510000   
NumRotBonds         5.000000    7.000000    6.000000    7.000000    2.000000   
NumAromaticRings    3.000000    3.000000    2.000000    4.000000    4.000000   

                  Molecule_6  Molecule_7  Molecule_8  Molecule_9  Molecule_10  \
MolWt             530.456000  464.831000   442.49100  398.482000   346.350000   
MolLogP             5.190380    5.549700     1.68624    3.334940     2.329400   
EStateSum          79.888889   94.36

# Classification model

In [306]:
# convert to 0s and 1s

# we are testing the variable val
# param is the column index
# i = every kinase
def classify(param, val, i):
    # Grab x-values from transposed_df (row (0-2), columns 1 to 60)
    x_values = pd.to_numeric(transposed_df.iloc[param, 1:60], errors='coerce').values.reshape(-1, 1)
    # Grab y-values from Kd (row i, columns 4 to 64)
    y_values = pd.to_numeric(Kd.iloc[i, 4:64], errors='coerce').values
    y_values = np.where(y_values == 10001, 1, 0)

    # Create a boolean mask: valid if neither x nor y is NaN and y != 10001
    mask = (~np.isnan(x_values.flatten())) & (~np.isnan(y_values)) & (y_values != 10001)

    x_filtered = x_values[mask]
    y_filtered = y_values[mask]

    # Train the Linear Regression model on the filtered data
    model = LinearRegression()
    model.fit(x_filtered, y_filtered)

    # Predictions for the training data
    y_pred = model.predict(x_filtered)

    # Predict y when x = 500 (convert 500 to a NumPy array and reshape it)
    x_new = np.array([val]).reshape(-1, 1)
    prediction = model.predict(x_new)
    return prediction

def classification_test(val, i):
    sum = 0
    for k in range(3):
      sum += classify(k, val, i)

    return sum/3

# Regression model

In [307]:
# convert to 0s and 1s
i = 1
def regress(param, val, i):
    # Grab x-values from transposed_df (row (0-2), columns 1 to 60)
    x_values = pd.to_numeric(transposed_df.iloc[param, 1:60], errors='coerce').values.reshape(-1, 1)
    # Grab y-values from Kd (row i, columns 4 to 64)
    y_values = pd.to_numeric(Kd.iloc[i, 4:64], errors='coerce').values

    # valid if neither x nor y is NaN and y != 10001
    mask = (~np.isnan(x_values.flatten())) & (~np.isnan(y_values)) & (y_values != 10001)

    x_filtered = x_values[mask]
    y_filtered = y_values[mask]

    model = LinearRegression()
    model.fit(x_filtered, y_filtered)

    y_pred = model.predict(x_filtered)
    x_new = np.array([val]).reshape(-1, 1)
    return model.predict(x_new)

In [311]:
# param = 0-7
# val = value to be predicted
def predict(param, val):
    total_prediction = 0

    for j in range(len(models[param])):
        model = models[param][j]
        if model is not None:
            # Predict only if classification_test result <= 0.5
            if classification_test(val, j) <= 0.99:
                pred = regress(param, val, j)
                if pred <= 300:
                    total_prediction += 1
        else:
            print(f"No model for row {param} and column {j}")

    return total_prediction / 442

print(predict(1, 4))

0.00904977375565611


In [312]:
#TESTING
def test_model():
    for i in range(12):
        sum = 0
        for k in range(3):
            sum+=predict(k, transposed_df_test.iloc[k, i])
        print(f"Predicting for molecule {i} : {sum/8}")

test_model()

Predicting for molecule 0 : 0.024604072398190045
Predicting for molecule 1 : 0.002828054298642534
Predicting for molecule 2 : 0.004524886877828055
Predicting for molecule 3 : 0.004807692307692308
Predicting for molecule 4 : 0.010746606334841629
Predicting for molecule 5 : 0.003393665158371041
Predicting for molecule 6 : 0.006787330316742082
Predicting for molecule 7 : 0.02177601809954751
Predicting for molecule 8 : 0.0053733031674208145
Predicting for molecule 9 : 0.016402714932126698
Predicting for molecule 10 : 0.004807692307692308
Predicting for molecule 11 : 0.006787330316742082
